In [ ]:
import os

print(os.environ.keys())

if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth vllm
# Install latest Hugging Face for Gemma-3!
!pip install --no-deps git+https://github.com/huggingface/transformers@v4.49.0-Gemma-3

In [ ]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 8,           # Larger = higher accuracy, but might overfit
    lora_alpha = 8,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

In [ ]:
from unsloth import FastModel
import torch
max_seq_length = 1024

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-1b-it",
    max_seq_length = max_seq_length, # Choose any for long context!
    load_in_4bit = False,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

In [ ]:
from datasets import load_dataset

# Load CSV into Hugging Face dataset
dataset = load_dataset("csv", data_files="fr-train-dataset2.csv")

# Access the split (train by default)
train_dataset = dataset["train"]

# Preview
print(train_dataset[0])
train_dataset

In [ ]:
train_dataset[0]["Address"]

In [ ]:
# Define columns to exclude from tgt
exclude_cols = ["Address", "Street_Name_old"]  # 👈 Add any column to ignore here

# Determine tgt columns
tgt_cols = [col for col in train_dataset.column_names if col not in exclude_cols]
src_col = "Address"


# Transform
def transform(example):
    return {
        "question": example[src_col],
        "answer": {col: example[col] for col in tgt_cols}
    }

dataset_refined = train_dataset.map(transform)

# Preview
print(dataset_refined[0])

In [ ]:
dataset_refined[0]["answer"]

In [ ]:
dataset_refined[0]["question"]